# Combine SCHISM output

This is the prototype for combining the multi-core output of SCHISM to one xarray datatet. This includes the construction of the grid references. It replaces autocombine_MPI_elfe.pl.
The example model used is the one of Jigsaw Setup Notebook.

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import glob
import pandas as pd
import xarray as xr
import numpy as np
import itertools
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
folder='/Users/brey/SCHISM/test_grid/'

## Read the global element index distribution to the cores  

In [ ]:
gtol = glob.glob(folder+'outputs/global*')

In [ ]:
gindx = pd.read_csv(gtol[0],header=None,delim_whitespace=True)

In [ ]:
gindx = gindx.set_index(gindx.columns[0]) # set the global index as index

In [ ]:
gindx.columns=['dist'] # rename the column to dist[ribution]

In [ ]:
gindx.head()

In [ ]:
#one can choose the elements on node 0
node0 = gindx.loc[gindx.dist == 0]

In [ ]:
node0.index

## Read the global node index distribution to the cores  

In [ ]:
gfiles = glob.glob(folder+'outputs/local*')
gfiles.sort()
gfiles

In [ ]:
#create a dict from filenames to identify parts in the dataframes below

keys=[]
for name in gfiles:
    keys.append('core{}'.format(name.split('/')[-1].split('_')[-1]))
keys

### Parsing the files

In [ ]:
# We read from the first file the header (it is the sama for all)
header = pd.read_csv(gfiles[0],header=None,nrows=1,delim_whitespace=True,names=['ns_global','ne_global','np_global','nvrt','nproc','ntracers','T','S','GEN','AGE','SED3D','EcoSim','ICM','CoSINE','Feco','TIMOR','FABM'])
#for i in range(1,len(gfiles)):
#    header = pd.concat([header,pd.read_csv(gfiles[i],header=None,nrows=1,delim_whitespace=True)])

In [ ]:
header = header.T # transpose for visual convenience 

In [ ]:
header

In [ ]:
#get the number of elems from all files
nels = []
for i in range(len(gfiles)):
    ne = pd.read_csv(gfiles[i],skiprows=2, header=None, nrows = 1)
    nels.append(ne.values.flatten()[0].astype(int))
nels

In [ ]:
#read and add them to pandas DataFrame 
frames=np.empty(len(gfiles),dtype=object)
for i in range(len(gfiles)):
    frames[i] = pd.read_csv(gfiles[i],skiprows=3,header=None, nrows=nels[i], names=['local','global_n'], delim_whitespace=True)

elems = pd.concat(frames,keys=keys)

elems.head()

In [ ]:
elems.max()

In [ ]:
#get the number of nodes from all files
nq = []
for i in range(len(gfiles)):
    nn = pd.read_csv(gfiles[i],skiprows=nels[i] + 3, header=None, nrows = 1)
    nq.append(nn.values.flatten()[0].astype(int))
nq

In [ ]:
#read and add them to pandas DataFrame
nframes=np.empty(len(gfiles),dtype=object)
for i in range(len(gfiles)):
    nframes[i] = pd.read_csv(gfiles[i],skiprows=nels[i] + 4,header=None, nrows=nq[i], names=['local','global_n'], delim_whitespace=True)
    
nodes = pd.concat(nframes,keys=keys)

nodes.head()

In [ ]:
nodes.max()

In [ ]:
#get the number of edges
nw = []
for i in range(len(gfiles)):
    nb = pd.read_csv(gfiles[i],skiprows=nels[i] + nq[i] + 4, header=None, nrows = 1)
    nw.append(nb.values.flatten()[0].astype(int))
nw

In [ ]:
#read and add them to pandas DataFrame
wframes=np.empty(len(gfiles),dtype=object)
for i in range(len(gfiles)):
    wframes[i] = pd.read_csv(gfiles[i],skiprows=nels[i] + nq[i] + 5,header=None, nrows=nw[i], names=['local','global_n'], delim_whitespace=True)
    
re = pd.concat(wframes,keys=keys)

re.head()

In [ ]:
re.max()

In [ ]:
#read secondary headers

h0 = pd.read_csv(gfiles[0],skiprows=nels[0] + nq[0] + nw[0] + 6, header=None, nrows = 1, delim_whitespace=True, names=['start_year','start_month','start_day','start_hour','utc_start'])

h1 = pd.read_csv(gfiles[0],skiprows=nels[0] + nq[0] + nw[0] + 7, header=None, nrows = 1, delim_whitespace=True, names = ['nrec','dtout','nspool','nvrt','kz','h0','h_s','h_c','theta_b','theta_f','ics'])


In [ ]:
ztots = ['ztot_'+str(i) for i in range(1,h1.loc[:,'kz']-1)]

ztots

In [ ]:
sigmas = ['sigma_'+str(i) for i in range(h1.loc[:,'nvrt'] - h1.loc[:,'kz'] + 1) ]
sigmas

In [ ]:
ztots + sigmas

In [ ]:
#read secondary header
h2 = pd.read_csv(gfiles[0],skiprows=nels[0] + nq[0] + nw[0] + 8, header=None, nrows = 1, delim_whitespace=True, names=ztots + sigmas)

In [ ]:
#combine headers
header2 = pd.concat([h0, h1, h2], axis=1)
#header2 = header2.T
header2

In [ ]:
#read lat/lon from all files
gframes=np.empty(len(gfiles),dtype=object)
for i in range(len(gfiles)):
    gframes[i] = pd.read_csv(gfiles[i],skiprows=nels[i] + nq[i] + nw[i] + 10, header=None, nrows = nq[i], delim_whitespace=True, names=['lon','lat','depth','kbp00'])
    
grid = pd.concat(gframes,keys=keys)

grid.head()

In [ ]:
#read tessellation 
eframes=np.empty(len(gfiles),dtype=object)
for i in range(len(gfiles)):
    eframes[i] = pd.read_csv(gfiles[i],skiprows=nels[i] + nq[i] + nw[i] + nq[i] + 10, header=None, nrows = nels[i], delim_whitespace=True, names=['type','a','b','c'])
    
tri = pd.concat(eframes,keys=keys)

tri.head()

## Check grid 

In [ ]:
for [a,b,c] in tri.loc['core0000',['a','b','c']].values[0:1]:
    print a,b,c
    [x1, y1] = grid.loc['core0000',['lon','lat']].values[a - 1]
    print(x1,y1)
    [x2, y2] = grid.loc['core0000',['lon','lat']].values[b - 1]
    print(x2,y2)
    [x3, y3] = grid.loc['core0000',['lon','lat']].values[c - 1]
    print(x3,y3)




In [ ]:
tri3 = tri.loc['core0000',['a','b','c']].values - 1 # for python index

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(grid.lon.values, grid.lat.values)
plt.triplot(grid.lon.values, grid.lat.values, tri3[:1], 'go-', lw=.5, markersize=5 )

w=np.array([[x1,y1],[x2,y2],[x3,y3]])
plt.scatter(w[:,0],w[:,1])
t1 = plt.Polygon(w[:3,:], color='g')
plt.gca().add_patch(t1)
plt.gca().annotate('0', xy=(w[0,0], w[0,1]),xytext=(w[0,0]+.01, w[0,1]+.01),xycoords='data',size=24)
plt.gca().annotate('1', xy=(w[1,0], w[1,1]),xytext=(w[1,0]+.01, w[1,1]+.01),xycoords='data',size=24)
plt.gca().annotate('2', xy=(w[2,0], w[2,1]),xytext=(w[2,0], w[2,1]+.01),xycoords='data',size=24)

plt.show()

## Read Netcdf output files

In [ ]:
files = glob.glob(folder+'outputs/schout_0*_*.nc')
files.sort()
files

In [ ]:
#store them in a list
out=[]
for i in range(len(keys)):
    ifiles = [f for f in files if '{}_'.format(i) in f]
    out.append(xr.open_mfdataset(ifiles))

In [ ]:
out[0] # example

## Read the combined NetCDFs with autocombine_MPI_elfe.pl.

Run "autocombine_MPI_elfe.pl 0 3" on the model folder. 

In [ ]:
cfiles = glob.glob(folder+'outputs/schout_0.nc') + glob.glob(folder+'outputs/schout_[!00]*.nc')
cfiles.sort()
cfiles

In [ ]:
gr = xr.open_mfdataset(cfiles[0], autoclose=True, drop_variables=['time'])

In [ ]:
var = xr.open_mfdataset(cfiles[1:], autoclose=True, drop_variables=gr.variables.keys())

In [ ]:
grf = xr.merge([gr,var])

In [ ]:
grf

## convert time to Timestamps

In [ ]:
date = header2.loc[:,['start_year','start_month','start_day','start_hour','utc_start']]

In [ ]:
date

In [ ]:
date.columns=['year','month','day','hour','utc'] # rename the columns

In [ ]:
date.year # check

In [ ]:
#set the start timestamp
sdate = pd.Timestamp(year=date.year.values, month=date.month.values, day=date.day.values, hour=date.hour.values, tz=date.utc.values)
sdate

In [ ]:
out[0].time.values # output times in seconds

In [ ]:
#get times as timestamps
times = pd.to_datetime(out[0].time.values, unit='s',
                   origin=sdate)
times

## Get the grid (nodes) indices

Since there are ghost nodes due to the mpi split we need to sort out the duplicates. This is done with the global_n index which would be the same for repeat nodes.

### Find duplicates

In [ ]:
nodes.global_n.duplicated(keep=False)

#### Check

In [ ]:
nodes.loc['core0000', 10] # choose one from the above True nodes to check

In [ ]:
nodes.loc[nodes.global_n==13] # find the duplicate indices equal to the global_n

It seems that the node 13 is common to core0000 and core00002

In [ ]:
grid.loc['core0000',10], grid.loc['core0002',46] # grid nodes are the same

In [ ]:
np.array_equal(out[0].elev[:,10].values, out[2].elev[:,46].values) # Elevation data are the same

In [ ]:
nodes.global_n.min(), nodes.global_n.max()# index range of nodes

In [ ]:
cnodes = nodes.global_n.drop_duplicates() # drop duplicate global nodes and store the values to an array

In [ ]:
cnodes.values

In [ ]:
cnodes.min(), cnodes.max(), cnodes.size # check

## Combine grid 

In [ ]:
#The duplicated nodes on thr grid dataFrame are...
grid.duplicated().sum()

In [ ]:
#which is the same as the duplicated nodes
nodes.global_n.duplicated().sum()

In [ ]:
dgrid = grid.drop_duplicates() # keep only one of the duplicates (the first)

In [ ]:
dgrid.index = dgrid.index.droplevel() # drop multi-index

In [ ]:
dgrid = dgrid.reset_index(drop=True) # reset index

In [ ]:
dgrid.shape #not that this is the same size as cnodes above -> check

In [ ]:
dgrid.index = cnodes.values - 1 # reindex based on the global index, -1 for the python convention

In [ ]:
dgrid.head()

In [ ]:
grd = dgrid.sort_index() #sort with the new index (that is the global_n)

In [ ]:
#reindex for final version
grd = grd.reset_index(drop=True)
#grd = grd.dropna()

In [ ]:
grd.head()

## Combine Tessellation

### for one core

In [ ]:
node0 = nodes.loc['core0000'].copy() # make a copy of the first core
node0 = node0.set_index('local') # reset the index using the local values (in essense set the index to start from 1...)
node0.head()

In [ ]:
nodes.max()

In [ ]:
tri0=tri.loc['core0000'].copy() # copy the tri dataframe for the first node
tri0.head()

In [ ]:
tri0.loc[:,'ga'] = node0.reindex(tri0['a'].values).values # create a new column where the local indices are replaced by the global ones 

In [ ]:
#repeat for all elements' nodes
tri0.loc[:,'gb'] = node0.reindex(tri0['b'].values).values
tri0.loc[:,'gc'] = node0.reindex(tri0['c'].values).values
tri0.head()

### for all cores

In [ ]:
#repeat for all cores
for key in keys:
    nod = nodes.loc[key].copy() # make a copy of the the core
    nod = nod.set_index('local') # reset the index using the local values (in essense set the index to start from 1...)
    tri.loc[key,'ga'] = nod.reindex(tri.loc[key,'a'].values).values
    tri.loc[key,'gb'] = nod.reindex(tri.loc[key,'b'].values).values
    tri.loc[key,'gc'] = nod.reindex(tri.loc[key,'c'].values).values

In [ ]:
tri.loc[:,'ga'] = tri.loc[:,'ga'].apply(pd.to_numeric(int)) # make integer
tri.loc[:,'gb'] = tri.loc[:,'gb'].apply(pd.to_numeric(int)) # make integer
tri.loc[:,'gc'] = tri.loc[:,'gc'].apply(pd.to_numeric(int)) # make integer

In [ ]:
tri.max()

In [ ]:
#keep the global indices
gtri = tri.loc[:,['ga','gb','gc']]
gtri.head()

### Check Orientation of elements

In [ ]:
#make a copy of gtri
gt = gtri.copy()
#gt.index = gt.index.droplevel() # drop multi-index
gt = gt.reset_index()
gt.head()

In [ ]:
# We expand the tri dataframe to include lon,lat columns
gt.loc[:,'x1'] = grd.loc[gt['ga'].values - 1 ,'lon'].values # -1 for python convention
gt.loc[:,'y1'] = grd.loc[gt['ga'].values - 1,'lat'].values

gt.loc[:,'x2'] = grd.loc[gt['gb'].values - 1,'lon'].values
gt.loc[:,'y2'] = grd.loc[gt['gb'].values - 1,'lat'].values

gt.loc[:,'x3'] = grd.loc[gt['gc'].values - 1,'lon'].values
gt.loc[:,'y3'] = grd.loc[gt['gc'].values - 1,'lat'].values

In [ ]:
#compute the value to establish orientation
gt['val'] = (gt['y2'] - gt['y1']) * (gt['x3'] - gt['x2']) - (gt['x2'] - gt['x1']) * (gt['y3'] - gt['y2'])

In [ ]:
gt.head()

In [ ]:
# Make all elements with anti-clockwise orientation
gt.loc[gt.val > 0] # clockwise that need to be reshuffled

In [ ]:
gt2 = gt.copy() # make a copy

#swap values 2,3 to change orientation
gt2.loc[gt2.val > 0, 'gb'] = gt.loc[gt.val > 0, 'gc'].values.astype(int)

gt2.loc[gt2.val > 0, 'gc'] = gt.loc[gt.val > 0, 'gb'].values.astype(int)

In [ ]:
#update the orientation refs again to verify
gt2.loc[:,'x1'] = grd.loc[gt2['ga'].values - 1,'lon'].values
gt2.loc[:,'y1'] = grd.loc[gt2['ga'].values - 1,'lat'].values

gt2.loc[:,'x2'] = grd.loc[gt2['gb'].values - 1,'lon'].values
gt2.loc[:,'y2'] = grd.loc[gt2['gb'].values - 1,'lat'].values

gt2.loc[:,'x3'] = grd.loc[gt2['gc'].values - 1,'lon'].values
gt2.loc[:,'y3'] = grd.loc[gt2['gc'].values - 1,'lat'].values

gt2['val'] = (gt2['y2'] - gt2['y1']) * (gt2['x3'] - gt2['x2']) - (gt2['x2'] - gt2['x1']) * (gt2['y3'] - gt2['y2'])

In [ ]:
gt2.loc[gt2.val > 0] #check

In [ ]:
# Now we need to put them in order based on the global index in elems
#gt2.index = elems.global_n.values # we set the index equal to the global_n column

In [ ]:
#gt2 = gt2.sort_index()
#gt2.head()

### Check grid (plot)

In [ ]:
tri3 = tri.xs(['ga','gb','gc'], axis=1, drop_level=True).values
tri3 = tri3 - 1 # python index

In [ ]:
tri3

In [ ]:
tri3.max()

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(grd.lon.values, grd.lat.values)
plt.triplot(grd.lon.values, grd.lat.values, tri3, 'go-', lw=.5, markersize=5 )


plt.show()

### Sort elements

In [ ]:
gt3 = tri.loc[:,['ga','gb','gc']].copy() # make a copy
gt3.index = gt3.index.droplevel() # drop multi-index
gt3 = gt3.reset_index(drop=True)

In [ ]:
# Now we need to put them in order based on the global index in elems
gt3.index = elems.global_n.values # we set the index equal to the global_n column

In [ ]:
gt3 = gt3.sort_index() #sort them

In [ ]:
#add nan column in place of the fourth node. This needs to be tested for quadrilaterals
gt3['gd']=np.nan

In [ ]:
gt3.head()

In [ ]:
gt3 = gt3.reset_index() # reset to add more columns without problems

In [ ]:
## Add mean x, y of the elememts. To be used in the output
gt3['x1'] = grd.loc[gt3['ga'].values - 1, 'lon'].values #lon of the index, -1 for python convention
gt3['y1'] = grd.loc[gt3['ga'].values - 1, 'lat'].values #lat of the index
gt3['x2'] = grd.loc[gt3['gb'].values - 1, 'lon'].values
gt3['y2'] = grd.loc[gt3['gb'].values - 1, 'lat'].values
gt3['x3'] = grd.loc[gt3['gc'].values - 1, 'lon'].values
gt3['y3'] = grd.loc[gt3['gc'].values - 1, 'lat'].values


gt3['xc'] =  gt3[['x1', 'x2', 'x3']].mean(axis=1) #mean lon of the element
gt3['yc'] =  gt3[['y1', 'y2', 'y3']].mean(axis=1)
gt3.head()

In [ ]:
## min kbe
gt3['kbe1'] = grd.loc[gt3['ga'] - 1,'kbp00'].values
gt3['kbe2'] = grd.loc[gt3['gb'] - 1,'kbp00'].values
gt3['kbe3'] = grd.loc[gt3['gc'] - 1,'kbp00'].values
#gt3['kbe4'] = grd.loc[gt3['gd'],'kbp00'].values

gt3['kbe'] = gt3[['kbe1', 'kbe2', 'kbe3']].min(axis=1)

In [ ]:
gt3 = gt3.set_index('index') # set index back 

In [ ]:
gt34 = gt3.loc[:,['ga','gb','gc','gd']].values # SCHISM_hgrid_face_nodes

In [ ]:
np.array_equal(gt3.values[:,:3],grf.SCHISM_hgrid_face_nodes.values[:,:3]) # check with autocombine_MPI_elfe.pl values

## Grid's Edges

In [ ]:
re.head() #indices for edges

In [ ]:
edges = re.loc[re.global_n.drop_duplicates().index] # keep only one of the duplicates

In [ ]:
edges.head()

In [ ]:
edges.shape

Apparently it goes [b,c], [c,a], [a,b] for some reason ?

In [ ]:
edgs=[]
for key in keys:
    for [ga,gb,gc] in tri.loc[key,['ga','gb','gc']].values:
        edgs.append([gb,gc])
        edgs.append([gc,ga])
        edgs.append([ga,gb])
        
edgs = np.array(edgs)
print edgs.shape
edgs

In [ ]:
edgs = pd.DataFrame(edgs)#make a pandas DataFrame
edgs.head()

In [ ]:
edgs.shape, edges.global_n.values.shape # difference due to duplicates

In [ ]:
edgs1 = edgs.apply(sorted, axis=1).drop_duplicates()
print edgs1.shape # correct shape
edgs1.head()

In [ ]:
# Now we need to put them in order based on the global index in edges
edgs1.index = edges.global_n.values # we set the index equal to the global_n column

In [ ]:
# .. and we sort
edgs1 = edgs1.sort_index()

In [ ]:
np.array_equal(edgs1.apply(sorted, axis=1).values, np.sort(grf.SCHISM_hgrid_edge_nodes.values)) # they are the same, but the orientation?

In [ ]:
np.array_equal(edgs1.values, grf.SCHISM_hgrid_edge_nodes.values) # they are the same, but the orientation is wrong?

In [ ]:
np.argwhere(edgs1.values != grf.SCHISM_hgrid_edge_nodes.values).shape #in many locations

### Start again with one core

In [ ]:
edgs0=[]
for [ga,gb,gc] in tri.loc['core0000',['ga','gb','gc']].values:
        edgs0.append([gb,gc])
        edgs0.append([gc,ga])
        edgs0.append([ga,gb])
        
edgs0 = np.array(edgs0)

edgs0 = pd.DataFrame(edgs0)#make a pandas DataFrame
edgs0.head()

In [ ]:
edgs0.shape, re.loc['core0000'].shape # different because of duplication

We see that the global edge index is less than the edges because there are duplicates in the form of [a, b],[b, a].

In [ ]:
idsd = edgs0[edgs0.apply(sorted, axis=1).duplicated()].index.values # find the duplicates

In [ ]:
edges01 = edgs0.drop(idsd) #drop them 

In [ ]:
#apply the global index
edges01.index = re.loc['core0000','global_n'].values

In [ ]:
edges01.head()

In [ ]:
edges01.loc[:,2] = grf.SCHISM_hgrid_edge_nodes.values[edges01.index.values-1,0] # add to the dataframe the correct results for comparison
edges01.loc[:,3] = grf.SCHISM_hgrid_edge_nodes.values[edges01.index.values-1,1]

In [ ]:
#compare with the correct solution
edges01[edges01[0] != edges01[2]] # only a few differences

### For all cores

In [ ]:
edk=[]
for key in keys:
    eds=[]
    for [ga,gb,gc] in tri.loc[key,['ga','gb','gc']].values:
        eds.append([gb,gc])
        eds.append([gc,ga])
        eds.append([ga,gb])
        
    eds = np.array(eds)

    df = pd.DataFrame(eds)
    idsd = df[df.apply(sorted, axis=1).duplicated()].index.values # find the duplicates
    df_ = df.drop(idsd) #drop them 
    df_.index = re.loc[key,'global_n'].values
    
    edk.append(df_)#make a pandas DataFrame

In [ ]:
edgs = pd.concat(edk) # We concatenate, however there are dublicate indices ...

In [ ]:
edgs.shape

In [ ]:
edgs[edgs.index.duplicated(keep=False)]

In [ ]:
#see https://stackoverflow.com/questions/13035764/remove-rows-with-duplicate-indices-pandas-dataframe-and-timeseries
edgs1 = edgs.reset_index().drop_duplicates(subset='index', keep='first').set_index('index') #drop duplicates 

In [ ]:
edgs1.shape, grf.SCHISM_hgrid_edge_nodes.shape #check

In [ ]:
edgs1 = edgs1.sort_index() #sort index 

In [ ]:
edgs1.loc[:,2] = grf.SCHISM_hgrid_edge_nodes.values[:,0] # add to the dataframe the correct results for comparison
edgs1.loc[:,3] = grf.SCHISM_hgrid_edge_nodes.values[:,1]

In [ ]:
edgs1.head()

In [ ]:
edgs1 = edgs1.reset_index() #reset index to add columns

In [ ]:
#mean x, y 
edgs1['x1'] = grd.loc[edgs1[0].values - 1, 'lon'].values #lon of the index, -1 for python convention
edgs1['y1'] = grd.loc[edgs1[0].values - 1, 'lat'].values #lat of the index
edgs1['x2'] = grd.loc[edgs1[1].values - 1, 'lon'].values
edgs1['y2'] = grd.loc[edgs1[1].values - 1, 'lat'].values
 
edgs1['xc'] =  edgs1[['x1', 'x2']].mean(axis=1) #mean of the edge index
edgs1['yc'] =  edgs1[['y1', 'y2']].mean(axis=1)
edgs1.head()

In [ ]:
edgs1[edgs1[0] != edgs1[2]] # still some problems with orientation

In [ ]:
## min bottom index
edgs1['kbs1'] = grd.loc[edgs1[0] - 1,'kbp00'].values
edgs1['kbs2'] = grd.loc[edgs1[1] - 1,'kbp00'].values

edgs1['kbs'] = edgs1[['kbs1', 'kbs2']].min(axis=1)

In [ ]:
edgs1.set_index('index') # set index again

In [ ]:
edgs1.head()

#### check the edges that have a problem

In [ ]:
er = edgs1[edgs1[0] != edgs1[2]]
er = er.reset_index()
er.head()

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(grd.lon.values, grd.lat.values)
plt.triplot(grd.lon.values, grd.lat.values, tri3, 'go-', lw=.5, markersize=5 )
for i in range(er.shape[0]):
    plt.plot([er.loc[i,'x1'],er.loc[i,'x2']],[er.loc[i,'y1'],er.loc[i,'y2']])

plt.show()

## Combine element-wise variables

Now create a full Dataset and fill them up with the appropriate values. We use the 'wed_dry' variable as example

In [ ]:
s = pd.Series(gindx.values.flatten()) # create a series with the elements node reference

In [ ]:
data = pd.concat([s] * times.shape[0], axis=1) # concatenate to the number of time steps

In [ ]:
data.columns=times # set columns names as the timestamps

In [ ]:
data.head() # view

In [ ]:
wd = data.copy() # make a copy for safery

In [ ]:
print wd.loc[data['2013-10-28 01:00:00'] == 0 ].shape, out[0].wetdry_elem.values.T.shape # check shapes

In [ ]:
for time in times: #all times
    for i in range(len(keys)): # all components
        wd.loc[data[time] == i] = out[i].wetdry_elem.values.T # wetdry_elem variable

In [ ]:
wd.head()

In [ ]:
wd.T.shape # check shape

### All variables

In [ ]:
def ecombine(var,out,data,times):
    wd = data.copy()
    for time in times: #all times
        for i in range(len(keys)): # all components
            wd.loc[data[time] == i] = out[i][var].values.T #

    return wd.T


In [ ]:
edic={}
for var in out[0].variables.keys():
    print out[0][var].name, out[0][var].dims, len(out[0][var].dims)
    if ('nSCHISM_hgrid_face' in out[0][var].dims) & (len(out[0][var].dims) == 2):
        wd = data.copy()
        for time in times: #all times
            for i in range(len(keys)): # all components
                wd.loc[data[time] == i] = out[i][var].values.T #
        vname = out[0][var].name
        edic[vname]=wd.T.values
        
    elif ('nSCHISM_hgrid_face' in out[0][var].dims) & ('two' in out[0][var].dims):
        wd = data.copy()
        for time in times: #all times
            for i in range(len(keys)): # all components
                wd.loc[data[time] == i] = out[i][var].values[:,:,0].T # wetdry_elem variable
        vx = wd.T
                
        wd = data.copy()
        for time in times: #all times
            for i in range(len(keys)): # all components
                wd.loc[data[time] == i] = out[i][var].values[:,:,1].T # wetdry_elem variable

        vy = wd.T

        vname = out[0][var].name
        edic[vname] = np.dstack([vx.values,vy.values])
        
    elif ('nSCHISM_hgrid_face' in out[0][var].dims) & ('nSCHISM_vgrid_layers' in out[0][var].dims):
        s=out[0][var].shape[2]
        ars=[]
        for l in range(s):
            wd = data.copy()
            for time in times: #all times
                for i in range(len(keys)): # all components
                    wd.loc[data[time] == i] = out[i][var].values[:,:,0].T # wetdry_elem variable

            ars.append(wd.T)

        vname = out[0][var].name
        edic[vname] = np.dstack([v.values for v in ars])



        

In [ ]:
np.array_equal(edic['wetdry_elem'],wd.T.values) # check

## Combine node-wise variables

As an example we do the 'elev' variable

In [ ]:
pout = out[0].elev.to_pandas().T
for i in range(1,len(keys)):
    pout = pd.concat([pout, out[i].elev.to_pandas().T])

In [ ]:
pout.head()

In [ ]:
pout = pout.reset_index(drop=True) # reset index

In [ ]:
pout = pout.drop(pout[grid.duplicated().values].index) # drop duplicate nodes
pout.head()

In [ ]:
pout.index = cnodes.values - 1 # reindex based on the global index -1 for the python convention

In [ ]:
pout.head()

In [ ]:
elev = pout.sort_index() #sort with the global index

In [ ]:
#reindex for final version
elev = elev.reset_index(drop=True)

In [ ]:
elev.columns = times # set time stamp 

In [ ]:
elev = elev.T # transpose to set time as index

In [ ]:
elev.head()

In [ ]:
sl = elev.loc['2013-10-28 04:00:00'].values

In [ ]:
sl.min(), sl.max(), sl.mean()

In [ ]:
#test
plt.figure(figsize=(12,10))
plt.gca().set_aspect('equal')
plt.tricontourf(grd.lon.values, grd.lat.values, tri3, sl, 50 )
plt.colorbar()
plt.show()

### All variables

In [ ]:
drop_mask = grid.duplicated().values # save the mask

In [ ]:
drop_mask.shape

In [ ]:
#Function for combining variables
def combine(ars, drop_mask, cnodes, times):
        pout = ars[0].to_pandas().T
        for f in ars[1:]:
            pout = pd.concat([pout, f.to_pandas().T])
        
        pout = pout.reset_index(drop=True) # reset index
        
        pout = pout.drop(pout[drop_mask].index) # drop duplicate nodes
        
        pout.index = cnodes.values - 1 # reindex based on the global index -1 for the python convention
        
        pout = pout.sort_index() #sort with the global index
        
        
        pout = pout.reset_index(drop=True)#reindex for final version
        
        pout.columns = times # set time stamp 
        
        return pout.T # transpose to set time as index

In [ ]:
vdic={}
for var in out[0].variables.keys():
    print out[0][var].name, out[0][var].dims, len(out[0][var].dims)
    if ('nSCHISM_hgrid_node' in out[0][var].dims) & (len(out[0][var].dims) == 2):
        ars = [v[out[0][var].name] for v in out]
       
        v = combine(ars, drop_mask, cnodes, times)
        
        vdic[out[0][var].name] = v.values
        
    elif ('nSCHISM_hgrid_node' in out[0][var].dims) & ('two' in out[0][var].dims) & (len(out[0][var].dims) == 3):
        ars1 = [v[out[0][var].name][:,:,0] for v in out]
        vx = combine(ars1, drop_mask, cnodes, times)
        ars2 = [v[out[0][var].name][:,:,1] for v in out]
        vy = combine(ars2, drop_mask, cnodes, times)
        
        vname = out[0][var].name
        vdic[vname] = np.dstack([vx.values,vy.values])
        
    elif ('nSCHISM_hgrid_node' in out[0][var].dims) & ('nSCHISM_vgrid_layers' in out[0][var].dims) & (len(out[0][var].dims) == 3):
        s=out[0][var].shape[2]
        ars=[]
        for l in range(s):
            arsi = [v[out[0][var].name][:,:,l] for v in out]
            ars.append(combine(arsi, drop_mask, cnodes, times))

        vname = out[0][var].name
        vdic[vname] = np.dstack([v.values for v in ars])

    elif ('nSCHISM_hgrid_node' in out[0][var].dims) & ('nSCHISM_vgrid_layers' in out[0][var].dims) & ('two' in out[0][var].dims) & (len(out[0][var].dims) == 4):
        s=out[0][var].shape[2]
        ars=[]
        for l in range(s):
            arsx = [v[out[0][var].name][:,:,l,0] for v in out]
            vx = combine(arsx, drop_mask, cnodes, times)
            arsy = [v[out[0][var].name][:,:,l,1] for v in out]
            vy = combine(arsy, drop_mask, cnodes, times)
            ars.append(np.dstack([vx.values,vy.values]))
        
        vname = out[0][var].name
        vdic[vname] = np.stack([a for a in ars], axis=2) # stack correctly
        

In [ ]:
vdic.keys()

## Create output structure

### General properties

In [ ]:
sigms = header2.loc[:,sigmas].values.flatten() # get sigmas

In [ ]:
iwet_dry = 0  # defined by the user
ihgrid_id = -2147483647 # defined by user - 0,dummy_dim,ihgrid_id

### Xarray for element-based variables

In [ ]:
edic.keys()

In [ ]:
xrdic={}
for key in edic.iterkeys():
    xrdic.update({key:([x for x in out[0][key].dims],edic[key])})

In [ ]:
xe = xr.Dataset(xrdic,coords={u'time':times, u'sigma': sigms })

In [ ]:
xe

In [ ]:
#Set Attrs (this needs some automation)
xe.wetdry_elem.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'elem', 'data_vertical_center' : 'full', 'i23d' : 4, 'ivs' : 1}

### Xarray for node-based variables

In [ ]:
vdic.keys()

In [ ]:
xrdic={}
for key in vdic.iterkeys():
    xrdic.update({key:([x for x in out[0][key].dims],vdic[key])})

In [ ]:
xn = xr.Dataset(xrdic,coords={u'time':times, u'sigma': sigms })

In [ ]:
xn

In [ ]:
#Set Attrs
xn.salt.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'node', 'data_vertical_center' : 'full', 'i23d' : 2, 'ivs' : 1}
xn.zcor.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'node', 'data_vertical_center' : 'full', 'i23d' : 2, 'ivs' : 1}
xn.temp.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'node', 'data_vertical_center' : 'full', 'i23d' : 2, 'ivs' : 1}
xn.vertical_velocity.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'node', 'data_vertical_center' : 'full', 'i23d' : 2, 'ivs' : 1}
xn.hvel.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'node', 'data_vertical_center' : 'full', 'i23d' : 2, 'ivs' : 2}
xn.elev.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'node', 'data_vertical_center' : 'full', 'i23d' : 1, 'ivs' : 1}
xn.wind_speed.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'node', 'data_vertical_center' : 'full', 'i23d' : 1, 'ivs' : 2}
#xv.dahv.attrs = {'mesh' : 'SCHISM_hgrid', 'data_horizontal_center' : 'node', 'data_vertical_center' : 'full', 'i23d' : 1, 'ivs' : 2}

### Xarray for Grid variables

In [ ]:
header2 #remember ...

In [ ]:
header2.nvrt, header2.kz

In [ ]:
#compute cs
klev = np.arange(header2.kz,header2.nvrt+1)
k = klev-header2.kz.values


cs=np.zeros(k)

cs=(1-header2.theta_b.values)*np.sinh(header2.theta_f.values*sigms[k])/np.sinh(header2.theta_f.values)+ \
    header2.theta_b.values*(np.tanh(header2.theta_f.values*(sigms[k]+0.5))-np.tanh(header2.theta_f.values*0.5))/2/np.tanh(header2.theta_f.values*0.5)

cs

In [ ]:
xg = xr.Dataset({u'SCHISM_hgrid' : ([u'one'], [ihgrid_id]),
                 u'SCHISM_hgrid_face_nodes' : ([u'nSCHISM_hgrid_face', u'nMaxSCHISM_hgrid_face_nodes'], gt34),
                 u'SCHISM_hgrid_edge_nodes' : ([u'nSCHISM_hgrid_edge', u'two'], edgs1[[0,1]].values),
                 u'SCHISM_hgrid_node_x' : ([u'nSCHISM_hgrid_node'], grd.lon.values),
                 u'SCHISM_hgrid_node_y' : ([u'nSCHISM_hgrid_node'], grd.lat.values),
                 u'node_bottom_index' : ([u'nSCHISM_hgrid_node'], grd.kbp00.values),
                 u'SCHISM_hgrid_face_x' : ([u'nSCHISM_hgrid_face'], gt3.loc[:,'xc'].values),
                 u'SCHISM_hgrid_face_y' : ([u'nSCHISM_hgrid_face'], gt3.loc[:,'yc'].values),                 
                 u'ele_bottom_index': ([u'nSCHISM_hgrid_face'], gt3.kbe.values ),
                 u'SCHISM_hgrid_edge_x' : ([u'nSCHISM_hgrid_edge'], edgs1['xc'].values),
                 u'SCHISM_hgrid_edge_y' : ([u'nSCHISM_hgrid_edge'], edgs1['yc'].values ),
                 u'edge_bottom_index' : ([u'nSCHISM_hgrid_edge'], edgs1.kbs.values),                 
                 u'depth': ([u'nSCHISM_hgrid_node'], grd.depth.values),
                 u'dry_value_flag' : ([u'one'], [iwet_dry]),
                 u'coordinate_system_flag' : ([u'one'], header2.loc[:,'ics'].values),                
                 u'minimum_depth': ([u'one'], header2.loc[:,'h0'].values),
                 u'sigma_h_c' : ([u'one'], header2.loc[:,'h_c'].values),
                 u'sigma_theta_b': ([u'one'], header2.loc[:,'theta_b'].values),
                 u'sigma_theta_f' : ([u'one'], header2.loc[:,'theta_f'].values),
                 u'sigma_maxdepth' : ([u'one'], header2.loc[:,'h_s'].values),
                 u'Cs' : (['sigma'], cs)},
                     coords={u'time':times, u'sigma': sigms })

In [ ]:
xg

### Set attrs

In [ ]:
#Choose attrs
if header2.ics.values == 1:
    lat_coord_standard_name = 'projection_x_coordinate'
    lon_coord_standard_name = 'projection_y_coordinate'
    x_units = 'm'
    y_units = 'm'
    lat_str_len = 23
    lon_str_len = 23
else:
    lat_coord_standard_name = 'latitude'
    lon_coord_standard_name = 'longitude'
    x_units = 'degrees_north'
    y_units = 'degrees_east'
    lat_str_len = 8
    lon_str_len = 9

In [ ]:
#set Attrs
xg.SCHISM_hgrid_node_x.attrs = {'long_name' : 'node x-coordinate', 'standard_name' : lon_coord_standard_name , 'units' : x_units, 'mesh' : 'SCHISM_hgrid'}

xg.SCHISM_hgrid_node_y.attrs = {'long_name' : 'node y-coordinate', 'standard_name' : lat_coord_standard_name , 'units' : y_units, 'mesh' : 'SCHISM_hgrid'}

xg.depth.attrs = {'long_name' : 'Bathymetry', 'units' : 'meters', 'positive' : 'down', 'mesh' : 'SCHISM_hgrid', 'location' : 'node'}

xg.sigma_h_c.attrs = {'long_name' : 'ocean_s_coordinate h_c constant', 'units' : 'meters', 'positive' : 'down'}

xg.sigma_theta_b.attrs = {'long_name' : 'ocean_s_coordinate theta_b constant'}

xg.sigma_theta_f.attrs = {'long_name' : 'ocean_s_coordinate theta_f constant'}

xg.sigma_maxdepth.attrs = {'long_name' : 'ocean_s_coordinate maximum depth cutoff (mixed s over z bound...', 'units' : 'meters', 'positive' : 'down'}

xg.Cs.attrs = {'long_name' : 'Function C(s) at whole levels', 'positive' : 'up' }

xg.dry_value_flag.attrs = {'values' : '0: use last-wet value; 1: use junk'}

xg.SCHISM_hgrid_face_nodes.attrs = {'long_name' : 'Horizontal Element Table', 'cf_role' : 'face_node_connectivity' , 'start_index' : 1}

xg.SCHISM_hgrid_edge_nodes.attrs = {'long_name' : 'Map every edge to the two nodes that it connects', 'cf_role' : 'edge_node_connectivity' , 'start_index' : 1}

xg.SCHISM_hgrid_edge_x.attrs = {'long_name' : 'x_coordinate of 2D mesh edge' , 'standard_name' : lon_coord_standard_name, 'units' : 'm', 'mesh' : 'SCHISM_hgrid'}

xg.SCHISM_hgrid_edge_y.attrs = {'long_name' : 'y_coordinate of 2D mesh edge' , 'standard_name' : lat_coord_standard_name, 'units' : 'm', 'mesh' : 'SCHISM_hgrid'}

xg.SCHISM_hgrid_face_x.attrs = {'long_name' : 'x_coordinate of 2D mesh face' , 'standard_name' : lon_coord_standard_name, 'units' : 'm', 'mesh' : 'SCHISM_hgrid'}

xg.SCHISM_hgrid_face_y.attrs = {'long_name' : 'y_coordinate of 2D mesh face' , 'standard_name' : lat_coord_standard_name, 'units' : 'm', 'mesh' : 'SCHISM_hgrid'}

xg.SCHISM_hgrid.attrs = {'long_name' : 'Topology data of 2d unstructured mesh',
                           'topology_dimension' : 2,
                           'cf_role' : 'mesh_topology',
                           'node_coordinates' : 'SCHISM_hgrid_node_x SCHISM_hgrid_node_y',
                           'face_node_connectivity' : 'SCHISM_hgrid_face_nodes',
                           'edge_coordinates' : 'SCHISM_hgrid_edge_x SCHISM_hgrid_edge_y',
                           'face_coordinates' : 'SCHISM_hgrid_face_x SCHISM_hgrid_face_y',
                           'edge_node_connectivity' : 'SCHISM_hgrid_edge_nodes'
                          }

xg.node_bottom_index.attrs = {'long_name' : 'bottom level index at each node' , 'units' : 'non-dimensional', 'mesh' : 'SCHISM_hgrid', 'location' : 'node',
    'start_index' : 1}

xg.ele_bottom_index.attrs = {'long_name' : 'bottom level index at each element' , 'units' : 'non-dimensional', 'mesh' : 'SCHISM_hgrid', 'location' : 'elem',
    'start_index' : 1}

xg.edge_bottom_index.attrs = {'long_name' : 'bottom level index at each edge' , 'units' : 'non-dimensional', 'mesh' : 'SCHISM_hgrid', 'location' : 'edge',
    'start_index' : 1}

### merge them

In [ ]:
dat = xr.merge([xg,xe,xn])

In [ ]:
dat.attrs = {'Conventions': 'CF-1.0, UGRID-1.0', 'title': 'SCHISM Model output', 'source': 'SCHISM model output version v10', 'references': 'http://ccrm.vims.edu/schismweb/',
             'history': 'created by pyPoseidon', 'comment': 'SCHISM Model output', 'type': 'SCHISM Model output', 'VisIT_plugin': 'https://schism.water.ca.gov/library/-/document_library/view/3476283' }

In [ ]:
dat

## Save to netcdf

In [ ]:
dat.to_netcdf(folder+'outputs/test.nc')

### test read

In [ ]:
ct = xr.open_mfdataset(folder+'outputs/test.nc')

In [ ]:
ct

In [ ]:
#compare to autocombine_MPI_elfe.pl values
for key in grf.variables.keys():
     if not grf[key].equals(ct[key]): print key

### Address the Failed comparisons

In [ ]:
# Values for SCHISM_hgrid_face_x check within machine accuracy
np.max(np.abs(grf.SCHISM_hgrid_face_x.values-ct.SCHISM_hgrid_face_x.values))

In [ ]:
# Values for SCHISM_hgrid_face_y check within machine accuracy
np.max(np.abs(grf.SCHISM_hgrid_face_y.values-ct.SCHISM_hgrid_face_y.values))

In [ ]:
# Values for SCHISM_hgrid_edge_x check within machine accuracy
np.max(np.abs(grf.SCHISM_hgrid_edge_x.values-ct.SCHISM_hgrid_edge_x.values))

In [ ]:
# Values for SCHISM_hgrid_edge_x check within machine accuracy
np.max(np.abs(grf.SCHISM_hgrid_edge_y.values-ct.SCHISM_hgrid_edge_y.values))

In [ ]:
# Values for SCHISM_hgrid_node_x check within machine accuracy
np.max(np.abs(grf.SCHISM_hgrid_node_x.values-ct.SCHISM_hgrid_node_x.values))

In [ ]:
# Values for SCHISM_hgrid_node_x check within machine accuracy
np.max(np.abs(grf.SCHISM_hgrid_node_y.values-ct.SCHISM_hgrid_node_y.values))

In [ ]:
# Values for theta_b check within machine accuracy
np.max(np.abs(grf.sigma_theta_f.values-ct.sigma_theta_f.values))

In [ ]:
# Values for zcor check within machine accuracy
b1 = np.ma.log(grf.zcor.values) #mask Inf values
b2 = np.ma.log(ct.zcor.values)
np.max(np.abs(b1-b2))

In [ ]:
# The above problem with the edges orientation for some elements
np.argwhere(grf.SCHISM_hgrid_edge_nodes.values!=ct.SCHISM_hgrid_edge_nodes.values)